In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 32
tree_depth = 8
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [3]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [4]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [5]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [6]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.180998802185059 | KNN Loss: 6.227673530578613 | BCE Loss: 1.9533252716064453
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.19716739654541 | KNN Loss: 6.2274298667907715 | BCE Loss: 1.9697372913360596
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.151121139526367 | KNN Loss: 6.227524757385254 | BCE Loss: 1.9235968589782715
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.188850402832031 | KNN Loss: 6.2274041175842285 | BCE Loss: 1.9614464044570923
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.152887344360352 | KNN Loss: 6.227034568786621 | BCE Loss: 1.9258532524108887
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.164999008178711 | KNN Loss: 6.226687431335449 | BCE Loss: 1.9383113384246826
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.155362129211426 | KNN Loss: 6.226534366607666 | BCE Loss: 1.9288280010223389
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.13614273071289 | KNN Loss: 6.226588726043701 | BCE Loss: 1.90955

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 5.934737205505371 | KNN Loss: 4.850061893463135 | BCE Loss: 1.0846753120422363
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 5.811098098754883 | KNN Loss: 4.706234455108643 | BCE Loss: 1.1048638820648193
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 5.732649326324463 | KNN Loss: 4.618710994720459 | BCE Loss: 1.113938331604004
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 5.6056976318359375 | KNN Loss: 4.51455545425415 | BCE Loss: 1.0911420583724976
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 5.611255645751953 | KNN Loss: 4.4998064041137695 | BCE Loss: 1.1114490032196045
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 5.560354709625244 | KNN Loss: 4.41553258895874 | BCE Loss: 1.1448222398757935
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 5.4366607666015625 | KNN Loss: 4.3242902755737305 | BCE Loss: 1.112370491027832
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 5.374316692352295 | KNN Loss: 4.267876148223877 | BCE Loss

Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 4.933305263519287 | KNN Loss: 3.8630523681640625 | BCE Loss: 1.070252776145935
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 4.93026876449585 | KNN Loss: 3.8572144508361816 | BCE Loss: 1.073054313659668
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 4.902609825134277 | KNN Loss: 3.868712902069092 | BCE Loss: 1.0338969230651855
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 4.924625396728516 | KNN Loss: 3.8884031772613525 | BCE Loss: 1.036222219467163
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 4.907341003417969 | KNN Loss: 3.8555479049682617 | BCE Loss: 1.0517933368682861
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 4.910784721374512 | KNN Loss: 3.857971429824829 | BCE Loss: 1.0528134107589722
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 4.968305587768555 | KNN Loss: 3.8832643032073975 | BCE Loss: 1.0850410461425781
Epoch 22 / 500 | iteration 25 / 30 | Total Loss: 5.030717372894287 | KNN Loss: 3.936378002166748 | BCE Lo

Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 4.861330032348633 | KNN Loss: 3.83554744720459 | BCE Loss: 1.0257823467254639
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 4.830349445343018 | KNN Loss: 3.784989833831787 | BCE Loss: 1.04535973072052
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 4.912267208099365 | KNN Loss: 3.822322368621826 | BCE Loss: 1.089944839477539
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 4.833408355712891 | KNN Loss: 3.7764010429382324 | BCE Loss: 1.057007074356079
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 4.839536666870117 | KNN Loss: 3.7879533767700195 | BCE Loss: 1.0515832901000977
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 4.895272254943848 | KNN Loss: 3.8180091381073 | BCE Loss: 1.0772629976272583
Epoch 33 / 500 | iteration 15 / 30 | Total Loss: 4.921525001525879 | KNN Loss: 3.8660085201263428 | BCE Loss: 1.055516242980957
Epoch 33 / 500 | iteration 20 / 30 | Total Loss: 4.872917175292969 | KNN Loss: 3.8399155139923096 | BCE Loss: 1

Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 4.803697109222412 | KNN Loss: 3.750314950942993 | BCE Loss: 1.0533822774887085
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 4.808242321014404 | KNN Loss: 3.760650634765625 | BCE Loss: 1.0475915670394897
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 4.770204544067383 | KNN Loss: 3.7297310829162598 | BCE Loss: 1.040473461151123
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 4.796553611755371 | KNN Loss: 3.752020835876465 | BCE Loss: 1.0445325374603271
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 4.824166774749756 | KNN Loss: 3.776170492172241 | BCE Loss: 1.0479962825775146
Epoch 44 / 500 | iteration 5 / 30 | Total Loss: 4.785524845123291 | KNN Loss: 3.7503693103790283 | BCE Loss: 1.0351555347442627
Epoch 44 / 500 | iteration 10 / 30 | Total Loss: 4.7650604248046875 | KNN Loss: 3.7463302612304688 | BCE Loss: 1.0187301635742188
Epoch 44 / 500 | iteration 15 / 30 | Total Loss: 4.760532379150391 | KNN Loss: 3.742922306060791 | BCE 

...

Epoch 448 / 500 | iteration 0 / 30 | Total Loss: 4.736355781555176 | KNN Loss: 3.720778465270996 | BCE Loss: 1.0155770778656006
Epoch 448 / 500 | iteration 5 / 30 | Total Loss: 4.739109039306641 | KNN Loss: 3.7167129516601562 | BCE Loss: 1.0223963260650635
Epoch 448 / 500 | iteration 10 / 30 | Total Loss: 4.711991786956787 | KNN Loss: 3.691059112548828 | BCE Loss: 1.0209325551986694
Epoch 448 / 500 | iteration 15 / 30 | Total Loss: 4.701459884643555 | KNN Loss: 3.704157590866089 | BCE Loss: 0.9973020553588867
Epoch 448 / 500 | iteration 20 / 30 | Total Loss: 4.701648712158203 | KNN Loss: 3.671044111251831 | BCE Loss: 1.030604600906372
Epoch 448 / 500 | iteration 25 / 30 | Total Loss: 4.761867523193359 | KNN Loss: 3.7083561420440674 | BCE Loss: 1.0535115003585815
Epoch 449 / 500 | iteration 0 / 30 | Total Loss: 4.688438415527344 | KNN Loss: 3.6693243980407715 | BCE Loss: 1.0191140174865723
Epoch 449 / 500 | iteration 5 / 30 | Total Loss: 4.73390007019043 | KNN Loss: 3.678891181945801 | 

Epoch 458 / 500 | iteration 20 / 30 | Total Loss: 4.711792945861816 | KNN Loss: 3.7024762630462646 | BCE Loss: 1.0093164443969727
Epoch 458 / 500 | iteration 25 / 30 | Total Loss: 4.749874114990234 | KNN Loss: 3.71945858001709 | BCE Loss: 1.030415415763855
Epoch 459 / 500 | iteration 0 / 30 | Total Loss: 4.678637504577637 | KNN Loss: 3.655155658721924 | BCE Loss: 1.0234819650650024
Epoch 459 / 500 | iteration 5 / 30 | Total Loss: 4.727014541625977 | KNN Loss: 3.693755865097046 | BCE Loss: 1.0332589149475098
Epoch 459 / 500 | iteration 10 / 30 | Total Loss: 4.726995468139648 | KNN Loss: 3.689645290374756 | BCE Loss: 1.0373499393463135
Epoch 459 / 500 | iteration 15 / 30 | Total Loss: 4.734009742736816 | KNN Loss: 3.701765537261963 | BCE Loss: 1.0322442054748535
Epoch 459 / 500 | iteration 20 / 30 | Total Loss: 4.679590702056885 | KNN Loss: 3.6703453063964844 | BCE Loss: 1.0092453956604004
Epoch 459 / 500 | iteration 25 / 30 | Total Loss: 4.7523698806762695 | KNN Loss: 3.700995683670044 

Epoch 469 / 500 | iteration 10 / 30 | Total Loss: 4.683858871459961 | KNN Loss: 3.6704912185668945 | BCE Loss: 1.0133675336837769
Epoch 469 / 500 | iteration 15 / 30 | Total Loss: 4.695420265197754 | KNN Loss: 3.6804099082946777 | BCE Loss: 1.0150105953216553
Epoch 469 / 500 | iteration 20 / 30 | Total Loss: 4.741342544555664 | KNN Loss: 3.7153501510620117 | BCE Loss: 1.0259926319122314
Epoch 469 / 500 | iteration 25 / 30 | Total Loss: 4.7017364501953125 | KNN Loss: 3.69100022315979 | BCE Loss: 1.0107359886169434
Epoch 470 / 500 | iteration 0 / 30 | Total Loss: 4.734338283538818 | KNN Loss: 3.6815943717956543 | BCE Loss: 1.052743911743164
Epoch 470 / 500 | iteration 5 / 30 | Total Loss: 4.714425563812256 | KNN Loss: 3.6779732704162598 | BCE Loss: 1.0364524126052856
Epoch 470 / 500 | iteration 10 / 30 | Total Loss: 4.703989505767822 | KNN Loss: 3.6965994834899902 | BCE Loss: 1.0073901414871216
Epoch 470 / 500 | iteration 15 / 30 | Total Loss: 4.701154708862305 | KNN Loss: 3.666209697723

Epoch 480 / 500 | iteration 0 / 30 | Total Loss: 4.7608184814453125 | KNN Loss: 3.7304775714874268 | BCE Loss: 1.0303411483764648
Epoch 480 / 500 | iteration 5 / 30 | Total Loss: 4.711490631103516 | KNN Loss: 3.676882743835449 | BCE Loss: 1.0346076488494873
Epoch 480 / 500 | iteration 10 / 30 | Total Loss: 4.69797420501709 | KNN Loss: 3.6910603046417236 | BCE Loss: 1.006913661956787
Epoch 480 / 500 | iteration 15 / 30 | Total Loss: 4.758475303649902 | KNN Loss: 3.7306647300720215 | BCE Loss: 1.0278103351593018
Epoch 480 / 500 | iteration 20 / 30 | Total Loss: 4.7572455406188965 | KNN Loss: 3.742795705795288 | BCE Loss: 1.014449954032898
Epoch 480 / 500 | iteration 25 / 30 | Total Loss: 4.720937728881836 | KNN Loss: 3.6801161766052246 | BCE Loss: 1.0408214330673218
Epoch 481 / 500 | iteration 0 / 30 | Total Loss: 4.767848491668701 | KNN Loss: 3.7440948486328125 | BCE Loss: 1.0237535238265991
Epoch 481 / 500 | iteration 5 / 30 | Total Loss: 4.680918216705322 | KNN Loss: 3.698016405105591

Epoch 490 / 500 | iteration 20 / 30 | Total Loss: 4.692294120788574 | KNN Loss: 3.6814565658569336 | BCE Loss: 1.0108373165130615
Epoch 490 / 500 | iteration 25 / 30 | Total Loss: 4.7550048828125 | KNN Loss: 3.698802947998047 | BCE Loss: 1.0562021732330322
Epoch 491 / 500 | iteration 0 / 30 | Total Loss: 4.733845233917236 | KNN Loss: 3.694500684738159 | BCE Loss: 1.0393446683883667
Epoch 491 / 500 | iteration 5 / 30 | Total Loss: 4.711842060089111 | KNN Loss: 3.6803207397460938 | BCE Loss: 1.031521201133728
Epoch 491 / 500 | iteration 10 / 30 | Total Loss: 4.733510494232178 | KNN Loss: 3.692293643951416 | BCE Loss: 1.0412169694900513
Epoch 491 / 500 | iteration 15 / 30 | Total Loss: 4.728346824645996 | KNN Loss: 3.7061851024627686 | BCE Loss: 1.022161602973938
Epoch 491 / 500 | iteration 20 / 30 | Total Loss: 4.738253116607666 | KNN Loss: 3.701615810394287 | BCE Loss: 1.036637306213379
Epoch 491 / 500 | iteration 25 / 30 | Total Loss: 4.7347564697265625 | KNN Loss: 3.7237050533294678 |

In [7]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 1.4957e+00,  2.2320e+00,  3.2307e+00,  1.8412e+00,  4.1501e+00,
          1.0916e+00,  1.3062e+00,  2.3809e+00,  1.4414e+00,  2.5832e+00,
          2.7500e+00,  2.0658e+00,  8.5737e-01,  2.2502e+00,  1.5931e+00,
          2.1076e+00,  1.5929e+00,  3.8431e+00,  1.5590e+00,  2.7104e+00,
          1.8890e+00,  2.1590e+00,  1.8157e+00,  1.7746e+00,  2.1227e+00,
          2.1804e+00,  2.2866e+00,  1.7074e+00,  1.2637e+00,  7.7037e-01,
         -7.2234e-03,  1.2120e+00,  4.1476e-01,  1.2188e+00,  1.2754e+00,
          1.6367e+00,  1.2974e+00,  3.5390e+00,  8.6547e-01,  1.8028e+00,
          1.3451e+00, -7.5467e-01,  2.4334e-01,  2.4294e+00,  2.7278e+00,
          1.0045e+00, -5.0151e-01,  9.3344e-02,  1.2230e+00,  1.9194e+00,
          2.4503e+00, -1.1153e-01,  9.0962e-01,  3.8952e-01, -4.6239e-01,
          9.9071e-01,  1.9920e+00,  1.6375e+00,  1.2462e+00,  1.6857e+00,
          1.9570e-01,  6.7696e-01,  5.1394e-01,  1.1245e+00,  1.8090e+00,
          1.5869e+00, -1.8152e+00,  1.

In [8]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [12]:
dataset_ = [d[0].cpu() for d in dataset]

In [13]:
model = model.eval().cpu()
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 18.22it/s]


In [14]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [15]:
clusters = DBSCAN(eps=0.2, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [16]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [18]:
tensor_dataset = torch.stack(dataset_)

In [19]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [20]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [21]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [22]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [23]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [24]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [25]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [26]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [27]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [28]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [29]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [30]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [31]:
losses = []
accs = []
sparsity = []

In [32]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
Epoch: 00 | Batch: 000 / 027 | Total loss: 9.630 | Reg loss: 0.009 | Tree loss: 9.630 | Accuracy: 0.000000 | 0.58 sec/iter
Epoch: 00 | Batch: 001 / 027 | Total loss: 9.626 | Reg loss: 0.009 | Tree loss: 9.626 | Accuracy: 0.000000 | 0.434 sec/iter
Epoch: 00 | Batch: 002 / 027 | Total loss: 9.620 | Reg loss: 0.008 | Tree loss: 9.620 | Accuracy: 0.000000 | 0.387 sec/iter
Epoch: 00 | Batch: 003 / 027 | Total loss: 9.616 | Reg loss: 0.008 | Tree loss: 9.616 | Accuracy: 0.000000 | 0.365 sec/iter
Epoch: 00 | Batch: 004 / 027 | Total loss: 9.606 | Reg loss: 0.008 | Tree loss: 9.606 | Accuracy: 0.000000 | 0.351 sec/iter
Epoch: 00 | Batch: 005 / 027 | Total loss: 9.603 | Reg loss: 0.008 | Tree loss: 9.603 | Accuracy: 0.000000 | 0.343 sec/iter
Epoch: 00 | Batch: 006 / 027 | Total loss: 9.594 | Reg loss: 0.007 | Tree loss: 9.594 | Accuracy: 0.000000 | 0.346 sec/iter
Epoch: 00 | Batch:

Epoch: 02 | Batch: 008 / 027 | Total loss: 9.426 | Reg loss: 0.008 | Tree loss: 9.426 | Accuracy: 0.105469 | 0.328 sec/iter
Epoch: 02 | Batch: 009 / 027 | Total loss: 9.433 | Reg loss: 0.008 | Tree loss: 9.433 | Accuracy: 0.087891 | 0.328 sec/iter
Epoch: 02 | Batch: 010 / 027 | Total loss: 9.418 | Reg loss: 0.008 | Tree loss: 9.418 | Accuracy: 0.097656 | 0.329 sec/iter
Epoch: 02 | Batch: 011 / 027 | Total loss: 9.411 | Reg loss: 0.008 | Tree loss: 9.411 | Accuracy: 0.105469 | 0.329 sec/iter
Epoch: 02 | Batch: 012 / 027 | Total loss: 9.403 | Reg loss: 0.009 | Tree loss: 9.403 | Accuracy: 0.119141 | 0.329 sec/iter
Epoch: 02 | Batch: 013 / 027 | Total loss: 9.394 | Reg loss: 0.009 | Tree loss: 9.394 | Accuracy: 0.134766 | 0.329 sec/iter
Epoch: 02 | Batch: 014 / 027 | Total loss: 9.398 | Reg loss: 0.009 | Tree loss: 9.398 | Accuracy: 0.101562 | 0.329 sec/iter
Epoch: 02 | Batch: 015 / 027 | Total loss: 9.391 | Reg loss: 0.010 | Tree loss: 9.391 | Accuracy: 0.103516 | 0.329 sec/iter
Epoch: 0

Epoch: 04 | Batch: 017 / 027 | Total loss: 9.175 | Reg loss: 0.014 | Tree loss: 9.175 | Accuracy: 0.109375 | 0.331 sec/iter
Epoch: 04 | Batch: 018 / 027 | Total loss: 9.161 | Reg loss: 0.014 | Tree loss: 9.161 | Accuracy: 0.113281 | 0.331 sec/iter
Epoch: 04 | Batch: 019 / 027 | Total loss: 9.165 | Reg loss: 0.014 | Tree loss: 9.165 | Accuracy: 0.097656 | 0.331 sec/iter
Epoch: 04 | Batch: 020 / 027 | Total loss: 9.144 | Reg loss: 0.015 | Tree loss: 9.144 | Accuracy: 0.099609 | 0.331 sec/iter
Epoch: 04 | Batch: 021 / 027 | Total loss: 9.134 | Reg loss: 0.015 | Tree loss: 9.134 | Accuracy: 0.134766 | 0.331 sec/iter
Epoch: 04 | Batch: 022 / 027 | Total loss: 9.128 | Reg loss: 0.015 | Tree loss: 9.128 | Accuracy: 0.113281 | 0.332 sec/iter
Epoch: 04 | Batch: 023 / 027 | Total loss: 9.119 | Reg loss: 0.016 | Tree loss: 9.119 | Accuracy: 0.105469 | 0.332 sec/iter
Epoch: 04 | Batch: 024 / 027 | Total loss: 9.115 | Reg loss: 0.016 | Tree loss: 9.115 | Accuracy: 0.093750 | 0.331 sec/iter
Epoch: 0

Epoch: 06 | Batch: 026 / 027 | Total loss: 8.677 | Reg loss: 0.023 | Tree loss: 8.677 | Accuracy: 0.083333 | 0.328 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
Epoch: 07 | Batch: 000 / 027 | Total loss: 8.944 | Reg loss: 0.016 | Tree loss: 8.944 | Accuracy: 0.111328 | 0.33 sec/iter
Epoch: 07 | Batch: 001 / 027 | Total loss: 8.928 | Reg loss: 0.016 | Tree loss: 8.928 | Accuracy: 0.099609 | 0.329 sec/iter
Epoch: 07 | Batch: 002 / 027 | Total loss: 8.905 | Reg loss: 0.016 | Tree loss: 8.905 | Accuracy: 0.125000 | 0.329 sec/iter
Epoch: 07 | Batch: 003 / 027 | Total loss: 8.897 | Reg loss: 0.017 | Tree loss: 8.897 | Accuracy: 0.103516 | 0.329 sec/iter
Epoch: 07 | Batch: 004 / 027 | Total loss: 8.897 | Reg loss: 0.017 | Tree loss: 8.897 | Accuracy: 0.091797 | 0.329 sec/iter
Epoch: 07 | Batch: 005 

Epoch: 09 | Batch: 006 / 027 | Total loss: 8.447 | Reg loss: 0.022 | Tree loss: 8.447 | Accuracy: 0.105469 | 0.333 sec/iter
Epoch: 09 | Batch: 007 / 027 | Total loss: 8.458 | Reg loss: 0.022 | Tree loss: 8.458 | Accuracy: 0.082031 | 0.333 sec/iter
Epoch: 09 | Batch: 008 / 027 | Total loss: 8.404 | Reg loss: 0.023 | Tree loss: 8.404 | Accuracy: 0.089844 | 0.333 sec/iter
Epoch: 09 | Batch: 009 / 027 | Total loss: 8.377 | Reg loss: 0.023 | Tree loss: 8.377 | Accuracy: 0.111328 | 0.333 sec/iter
Epoch: 09 | Batch: 010 / 027 | Total loss: 8.342 | Reg loss: 0.023 | Tree loss: 8.342 | Accuracy: 0.099609 | 0.333 sec/iter
Epoch: 09 | Batch: 011 / 027 | Total loss: 8.327 | Reg loss: 0.024 | Tree loss: 8.327 | Accuracy: 0.121094 | 0.333 sec/iter
Epoch: 09 | Batch: 012 / 027 | Total loss: 8.309 | Reg loss: 0.024 | Tree loss: 8.309 | Accuracy: 0.125000 | 0.333 sec/iter
Epoch: 09 | Batch: 013 / 027 | Total loss: 8.277 | Reg loss: 0.024 | Tree loss: 8.277 | Accuracy: 0.105469 | 0.333 sec/iter
Epoch: 0

...

Epoch: 90 | Batch: 003 / 027 | Total loss: 3.029 | Reg loss: 0.033 | Tree loss: 3.029 | Accuracy: 0.105469 | 0.354 sec/iter
Epoch: 90 | Batch: 004 / 027 | Total loss: 3.026 | Reg loss: 0.033 | Tree loss: 3.026 | Accuracy: 0.082031 | 0.354 sec/iter
Epoch: 90 | Batch: 005 / 027 | Total loss: 3.037 | Reg loss: 0.033 | Tree loss: 3.037 | Accuracy: 0.126953 | 0.354 sec/iter
Epoch: 90 | Batch: 006 / 027 | Total loss: 2.983 | Reg loss: 0.033 | Tree loss: 2.983 | Accuracy: 0.101562 | 0.354 sec/iter
Epoch: 90 | Batch: 007 / 027 | Total loss: 2.987 | Reg loss: 0.033 | Tree loss: 2.987 | Accuracy: 0.105469 | 0.354 sec/iter
Epoch: 90 | Batch: 008 / 027 | Total loss: 3.000 | Reg loss: 0.033 | Tree loss: 3.000 | Accuracy: 0.119141 | 0.354 sec/iter
Epoch: 90 | Batch: 009 / 027 | Total loss: 3.001 | Reg loss: 0.033 | Tree loss: 3.001 | Accuracy: 0.105469 | 0.354 sec/iter
Epoch: 90 | Batch: 010 / 027 | Total loss: 2.973 | Reg loss: 0.033 | Tree loss: 2.973 | Accuracy: 0.123047 | 0.354 sec/iter
Epoch: 9

Epoch: 92 | Batch: 012 / 027 | Total loss: 2.961 | Reg loss: 0.033 | Tree loss: 2.961 | Accuracy: 0.113281 | 0.354 sec/iter
Epoch: 92 | Batch: 013 / 027 | Total loss: 2.939 | Reg loss: 0.033 | Tree loss: 2.939 | Accuracy: 0.113281 | 0.354 sec/iter
Epoch: 92 | Batch: 014 / 027 | Total loss: 2.882 | Reg loss: 0.033 | Tree loss: 2.882 | Accuracy: 0.123047 | 0.354 sec/iter
Epoch: 92 | Batch: 015 / 027 | Total loss: 2.927 | Reg loss: 0.033 | Tree loss: 2.927 | Accuracy: 0.087891 | 0.354 sec/iter
Epoch: 92 | Batch: 016 / 027 | Total loss: 2.908 | Reg loss: 0.033 | Tree loss: 2.908 | Accuracy: 0.097656 | 0.354 sec/iter
Epoch: 92 | Batch: 017 / 027 | Total loss: 2.956 | Reg loss: 0.033 | Tree loss: 2.956 | Accuracy: 0.123047 | 0.354 sec/iter
Epoch: 92 | Batch: 018 / 027 | Total loss: 2.900 | Reg loss: 0.033 | Tree loss: 2.900 | Accuracy: 0.101562 | 0.354 sec/iter
Epoch: 92 | Batch: 019 / 027 | Total loss: 2.902 | Reg loss: 0.033 | Tree loss: 2.902 | Accuracy: 0.082031 | 0.354 sec/iter
Epoch: 9

Epoch: 94 | Batch: 021 / 027 | Total loss: 2.906 | Reg loss: 0.033 | Tree loss: 2.906 | Accuracy: 0.099609 | 0.354 sec/iter
Epoch: 94 | Batch: 022 / 027 | Total loss: 2.881 | Reg loss: 0.033 | Tree loss: 2.881 | Accuracy: 0.113281 | 0.354 sec/iter
Epoch: 94 | Batch: 023 / 027 | Total loss: 2.867 | Reg loss: 0.033 | Tree loss: 2.867 | Accuracy: 0.097656 | 0.354 sec/iter
Epoch: 94 | Batch: 024 / 027 | Total loss: 2.889 | Reg loss: 0.033 | Tree loss: 2.889 | Accuracy: 0.117188 | 0.354 sec/iter
Epoch: 94 | Batch: 025 / 027 | Total loss: 2.862 | Reg loss: 0.033 | Tree loss: 2.862 | Accuracy: 0.109375 | 0.354 sec/iter
Epoch: 94 | Batch: 026 / 027 | Total loss: 2.712 | Reg loss: 0.033 | Tree loss: 2.712 | Accuracy: 0.166667 | 0.354 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
Epoch: 95 | Batch: 000

Epoch: 97 | Batch: 001 / 027 | Total loss: 3.044 | Reg loss: 0.032 | Tree loss: 3.044 | Accuracy: 0.109375 | 0.354 sec/iter
Epoch: 97 | Batch: 002 / 027 | Total loss: 3.063 | Reg loss: 0.032 | Tree loss: 3.063 | Accuracy: 0.101562 | 0.354 sec/iter
Epoch: 97 | Batch: 003 / 027 | Total loss: 3.054 | Reg loss: 0.032 | Tree loss: 3.054 | Accuracy: 0.093750 | 0.354 sec/iter
Epoch: 97 | Batch: 004 / 027 | Total loss: 3.054 | Reg loss: 0.032 | Tree loss: 3.054 | Accuracy: 0.089844 | 0.354 sec/iter
Epoch: 97 | Batch: 005 / 027 | Total loss: 3.000 | Reg loss: 0.032 | Tree loss: 3.000 | Accuracy: 0.107422 | 0.354 sec/iter
Epoch: 97 | Batch: 006 / 027 | Total loss: 3.048 | Reg loss: 0.032 | Tree loss: 3.048 | Accuracy: 0.095703 | 0.354 sec/iter
Epoch: 97 | Batch: 007 / 027 | Total loss: 2.966 | Reg loss: 0.032 | Tree loss: 2.966 | Accuracy: 0.119141 | 0.354 sec/iter
Epoch: 97 | Batch: 008 / 027 | Total loss: 2.990 | Reg loss: 0.032 | Tree loss: 2.990 | Accuracy: 0.123047 | 0.354 sec/iter
Epoch: 9

Epoch: 99 | Batch: 010 / 027 | Total loss: 3.009 | Reg loss: 0.032 | Tree loss: 3.009 | Accuracy: 0.126953 | 0.354 sec/iter
Epoch: 99 | Batch: 011 / 027 | Total loss: 2.961 | Reg loss: 0.032 | Tree loss: 2.961 | Accuracy: 0.103516 | 0.354 sec/iter
Epoch: 99 | Batch: 012 / 027 | Total loss: 2.970 | Reg loss: 0.032 | Tree loss: 2.970 | Accuracy: 0.095703 | 0.354 sec/iter
Epoch: 99 | Batch: 013 / 027 | Total loss: 2.915 | Reg loss: 0.032 | Tree loss: 2.915 | Accuracy: 0.101562 | 0.354 sec/iter
Epoch: 99 | Batch: 014 / 027 | Total loss: 2.969 | Reg loss: 0.032 | Tree loss: 2.969 | Accuracy: 0.105469 | 0.354 sec/iter
Epoch: 99 | Batch: 015 / 027 | Total loss: 2.939 | Reg loss: 0.032 | Tree loss: 2.939 | Accuracy: 0.115234 | 0.354 sec/iter
Epoch: 99 | Batch: 016 / 027 | Total loss: 2.939 | Reg loss: 0.032 | Tree loss: 2.939 | Accuracy: 0.091797 | 0.354 sec/iter
Epoch: 99 | Batch: 017 / 027 | Total loss: 2.899 | Reg loss: 0.032 | Tree loss: 2.899 | Accuracy: 0.109375 | 0.354 sec/iter
Epoch: 9

In [33]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [35]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 7.625


# Extract Rules

# Accumulate samples in the leaves

In [36]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 160


In [37]:
method = 'greedy'

In [38]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/miniconda3/envs/rambo/lib/python3.8/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [39]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))


============== Pattern 1 ==============
============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
============== Pattern 19 ==============
============== Pattern 20 ==============
============== Pattern 21 ==============
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 ==============
============== Pattern 25